## Labb 1.2

In [5]:
import pandas as pd 
from scipy.sparse import csr_matrix
from fuzzywuzzy import process


c:\Users\Pontus\Codes\Github\AI-Pontus-Berglund-OPA23-\venv\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [6]:
movies = "../data/ml-latest/movies.csv"
ratings = "../data/ml-latest/ratings.csv"
# Gets the data from movies and ratings
# Takes away some data we dont wanna use like timestamp on ratings
df_movies = pd.read_csv(movies, usecols = ["movieId", "title"], dtype={"movieId":"int32", "title":"str"})
df_ratings = pd.read_csv(ratings, usecols = ["userId", "movieId", "rating"], dtype={"userId":"int32", "movieId":"int32",  "rating":"float32"})

In [7]:
df_ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,110,4.0
2,1,158,4.0
3,1,260,4.5
4,1,356,5.0


In [35]:
# Filter out movies that have too few ratings and too low rating
total_ratings = df_ratings["movieId"].value_counts()
top_rated_movies = total_ratings[total_ratings >= 10000]
top_rated_movies_data = df_ratings[df_ratings["movieId"].isin(top_rated_movies.index)]
top_rated_movies_avg = top_rated_movies_data.groupby("movieId")["rating"].mean().reset_index()
top_rated_movies_avg_filtered = top_rated_movies_avg[top_rated_movies_avg["rating"] >= 3]
top_rated_movies_titles = top_rated_movies_avg_filtered.merge(df_movies, on="movieId")

# Take away users from the pivot tabel based on how many ratings
pivot_table_ratings = pd.pivot(top_rated_movies_data, values="rating", index="movieId", columns="userId").fillna(0)
user_counts = df_ratings["userId"].value_counts()
users_to_keep = user_counts[user_counts >= 1000].index
# Help from chatgpt
users_to_keep = [user for user in users_to_keep if user in pivot_table_ratings.columns]
pivot_table_ratings = pivot_table_ratings.loc[:, users_to_keep]

print(pivot_table_ratings)

userId   189614  48766   207216  175998  76618   230765  184775  236260  \
movieId                                                                   
1           4.0     3.5     4.0     3.0     5.0     5.0     3.0     5.0   
2           3.0     1.5     4.0     3.0     1.0     4.0     3.0     3.0   
3           0.0     3.0     3.0     0.0     1.5     4.0     2.0     2.5   
5           2.5     2.5     3.0     0.0     1.0     5.0     2.5     1.5   
6           4.0     4.0     4.0     3.5     3.0     4.0     5.0     4.0   
...         ...     ...     ...     ...     ...     ...     ...     ...   
187593      3.5     0.0     0.0     3.0     2.0     4.0     4.0     0.0   
195159      4.0     0.0     0.0     3.5     5.0     4.0     4.0     0.0   
202439      4.0     3.0     0.0     4.5     4.5     4.5     4.0     0.0   
204698      0.0     4.0     0.0     2.5     3.0     3.5     4.0     0.0   
207313      0.0     3.5     0.0     3.5     4.0     4.5     3.0     0.0   

userId   233891  214831 

In [36]:
# sparse of the pivot tabel 
sparse_matrix = csr_matrix(pivot_table_ratings.values)

print(sparse_matrix)

  (0, 0)	4.0
  (0, 1)	3.5
  (0, 2)	4.0
  (0, 3)	3.0
  (0, 4)	5.0
  (0, 5)	5.0
  (0, 6)	3.0
  (0, 7)	5.0
  (0, 8)	5.0
  (0, 9)	5.0
  (0, 10)	3.0
  (0, 12)	4.0
  (0, 13)	4.0
  (0, 14)	3.0
  (0, 15)	4.5
  (0, 16)	3.5
  (0, 17)	2.5
  (0, 18)	3.5
  (0, 19)	4.0
  (0, 20)	5.0
  (0, 21)	5.0
  (0, 23)	4.0
  (0, 24)	2.0
  (0, 25)	4.0
  (0, 26)	4.0
  :	:
  (839, 3566)	4.0
  (839, 3568)	3.0
  (839, 3574)	3.0
  (839, 3575)	4.0
  (839, 3578)	4.0
  (839, 3581)	4.0
  (839, 3582)	4.0
  (839, 3585)	3.5
  (839, 3587)	3.0
  (839, 3592)	4.5
  (839, 3594)	4.5
  (839, 3595)	3.5
  (839, 3596)	4.0
  (839, 3603)	2.5
  (839, 3610)	3.0
  (839, 3612)	4.0
  (839, 3624)	3.0
  (839, 3627)	4.5
  (839, 3628)	4.5
  (839, 3631)	3.5
  (839, 3635)	4.0
  (839, 3637)	4.0
  (839, 3639)	4.5
  (839, 3650)	4.5
  (839, 3651)	2.5


## Labb 1.3

In [29]:
# model for measure the nearest neighbors
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric= "cosine", algorithm="brute", n_neighbors = 5)

In [30]:
model_knn.fit(sparse_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [60]:

def recommender(data, model, n_recommendations):
    # Makes a temporary variable of whick movie user selects to compare with
    movie_name = input("Ange en film du gillar: ")

    # Trains the data in an knn model
    model.fit(data)

    # Uses fuzzywuzzy for help with finding the right title
    idx = process.extractOne(movie_name, df_movies["title"])[2]
    print("Vald film:", df_movies["title"][idx])
    

    # gets the data from a movie based on the index it has 
    selected_movie_data = data[idx]

    # Help from chatgpt, it measures the closest neighbors
    _, indices = model.kneighbors(selected_movie_data.reshape(1, -1), n_neighbors=n_recommendations + 1)

    # Loop that print out top 5 recommendations
    for i in range(1, n_recommendations + 1):  
        recommended_idx = indices[0][i]
        recommended_movie_title = df_movies.loc[recommended_idx, "title"]
        print(f"Rekommendation {i}: {recommended_movie_title}")


recommender(sparse_matrix, model_knn, 5)

Vald film: Enfer, L' (1994)
Rekommendation 1: Disclosure (1994)
Rekommendation 2: Blue Sky (1994)
Rekommendation 3: Bio-Dome (1996)
Rekommendation 4: Young Poisoner's Handbook, The (1995)
Rekommendation 5: Maverick (1994)
